# Data Cleaning
The prior data quality analysis revealed several data anomalies requiring attention. 

1. Cases with invalid values for subtlety and breast density.
2. Missing calcification type, calcification distribution, mass shape, and mass margins data. 
3. Categories that have different spelling, but same meaning. 

In this brief notebook, we address these issues, producing a clean dataset for analysis. Concretely, our task list is as follows:

| # | Task                                                                             |
|---|----------------------------------------------------------------------------------|
| 1 | Replace invalid values for breast density with NA for imputation.                |
| 2 | Set mass and calcification morphological values to NA where they do not   apply. |
| 3 | Remove cases with invalid values for subtlety                                    |
| 4 | Replace calcification types 'LUCENT_CENTER' with 'LUCENT_CENTERED', and 'PLEOMORPHIC-PLEOMORPHIC', with 'PLEOMORPHIC' |
| 5 | Impute missing values using Multiple Imputation by Chained Equations   (MICE)    |
| 6 | Convert data to the appropriate data types                                       |
| 7 | Conduct random inspection of imputations.                                        |
| 8 | Conduct final data quality analysis.                                             |

In [1]:
import os
if 'jbook' in os.getcwd():
    os.chdir(os.path.abspath(os.path.join("../..")))

import pandas as pd
import numpy as np

from bcd.data.prep.case import CaseImputer
pd.options.display.max_rows = 999

from bcd.data.dqa.case import MassCaseDQA, CalcCaseDQA
from bcd.data import CALC_VARIABLES, MASS_VARIABLES

In [2]:
FP_IN = "data/staged/case.csv"
FP_CLEAN_CASES = "data/clean/cases.csv"
FP_CLEAN_CALC =  "data/clean/calc_cases.csv"
FP_CLEAN_MASS =  "data/clean/mass_cases.csv"
CASES_TO_REMOVE = ['P_00710_RIGHT_mass_MLO_1', 'P_00710_RIGHT_mass_CC_1']

## Load and Save Original Data
First, let's load the data and create an interim dataset for processing.

In [3]:
# Load data and capture original version.
df_orig = pd.read_csv(FP_IN)
df_interim = df_orig

## Task 1: Breast Density
Replace invalid values for breast density with NA for downstream imputation.

In [4]:
# Set invalid values for breast_density to NA
df_interim['breast_density'] = df_interim['breast_density'].replace(0, np.NAN)

## Task 2: Morphology Not Applicable
Set morphological variables to NA where they do not apply. For instance mass shape doesn't apply to calcification cases.

In [5]:
# Set case and mass data to NOT APPLICABLE where appropriate.
df_interim.loc[df_interim['abnormality_type'] == 'calcification', 'mass_shape'] = 'NOT APPLICABLE'
df_interim.loc[df_interim['abnormality_type'] == 'calcification', 'mass_margins'] = 'NOT APPLICABLE'
df_interim.loc[df_interim['abnormality_type'] == 'mass', 'calc_type'] = 'NOT APPLICABLE'
df_interim.loc[df_interim['abnormality_type'] == 'mass', 'calc_distribution'] = 'NOT APPLICABLE'

## Task 3: Case Removal
Remove cases with invalid values for subtlety.

In [6]:
df_interim = df_interim.loc[~df_interim['case_id'].isin(CASES_TO_REMOVE)]
msg = f"A total of {df_orig.shape[0] - df_interim.shape[0]} rows were removed from the dataset, leaving {df_interim.shape[0]} rows."
print(msg)

A total of 2 rows were removed from the dataset, leaving 3566 rows.


## Task 4: Category Alignment
Align categories that have the same meaning, but differ in spelling only.

In [7]:
df_interim.loc[df_interim['calc_type'] == 'LUCENT_CENTER', 'calc_type'] = 'LUCENT_CENTERED'
df_interim.loc[df_interim['calc_type'] == 'ROUND_AND_REGULAR-LUCENT_CENTER-DYSTROPHIC', 'calc_type'] = 'ROUND_AND_REGULAR-LUCENT_CENTERED-DYSTROPHIC'
df_interim.loc[df_interim['calc_type'] == 'PUNCTATE-LUCENT_CENTER', 'calc_type'] = 'PUNCTATE-LUCENT_CENTERED'
df_interim.loc[df_interim['calc_type'] == 'VASCULAR-COARSE-LUCENT_CENTER-ROUND_AND_REGULAR-PUNCTATE', 'calc_type'] = 'VASCULAR-COARSE-LUCENT_CENTERED-ROUND_AND_REGULAR-PUNCTATE'
df_interim.loc[df_interim['calc_type'] == 'ROUND_AND_REGULAR-LUCENT_CENTER', 'calc_type'] = 'ROUND_AND_REGULAR-LUCENT_CENTERED'
df_interim.loc[df_interim['calc_type'] == 'LUCENT_CENTER-PUNCTATE', 'calc_type'] = 'LUCENT_CENTERED-PUNCTATE'
df_interim.loc[df_interim['calc_type'] == 'COARSE-ROUND_AND_REGULAR-LUCENT_CENTER', 'calc_type'] = 'COARSE-ROUND_AND_REGULAR-LUCENT_CENTERED'
df_interim.loc[df_interim['calc_type'] == 'ROUND_AND_REGULAR-LUCENT_CENTER-PUNCTATE', 'calc_type'] = 'ROUND_AND_REGULAR-LUCENT_CENTERED-PUNCTATE'
df_interim.loc[df_interim['calc_type'] == 'COARSE-LUCENT_CENTER', 'calc_type'] = 'COARSE-LUCENT_CENTERED'
df_interim.loc[df_interim['calc_type'] == 'PLEOMORPHIC-PLEOMORPHIC', 'calc_type'] = 'PLEOMORPHIC'

## Task 5: Impute Missing Values
Multiple Imputation by Chained Equations (MICE) is a robust, informative method of estimating missing values in datasets. The procedure imputes missing data through an iterative series of predictive models which estimate the value of missing data using the other variables in the dataset. For this, we'll use our CaseImputer which wraps scikit-learn's IterativeImputer implementation of MICE.

First, let's capture the missing values as we will inspect them after imputation.

In [8]:
# Grab rows with missing data
null_mask = df_interim.isnull().any(axis=1)
df_missing = df_interim[null_mask]
msg = f"There are {df_missing.shape[0]} rows (approximately {round(df_missing.shape[0] / df_orig.shape[0] * 100,1)}% of the dataset) with missing data in the dataset."
print(msg)

There are 525 rows (approximately 14.7% of the dataset) with missing data in the dataset.


In [9]:
imp = CaseImputer(random_state=5)
imp.fit(df=df_interim)
df_clean = imp.transform(df=df_interim)

With that, let's save the data.

## Task 6: Data Types

In [10]:
dtypes = {"patient_id": "object",
"breast_density": "int32",
"left_or_right_breast": "category",
"image_view": "category",
"abnormality_id": "int32",
"abnormality_type": "category",
"calc_type": "category",
"calc_distribution": "category",
"assessment": "int32",
"pathology": "category",
"subtlety": "int32",
"fileset": "object",
"mass_shape": "category",
"mass_margins": "category",
"case_id": "object",
"cancer": "bool",
}
dtypes_ordinal = {
"breast_density": "category",
"assessment": "category",
"subtlety": "category",
}
df_clean = df_clean.astype(dtypes)
df_clean = df_clean.astype(dtypes_ordinal)
df_clean.info()
df_clean.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3566 entries, 0 to 3565
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   patient_id            3566 non-null   object  
 1   breast_density        3566 non-null   category
 2   left_or_right_breast  3566 non-null   category
 3   image_view            3566 non-null   category
 4   abnormality_id        3566 non-null   int32   
 5   abnormality_type      3566 non-null   category
 6   calc_type             3566 non-null   category
 7   calc_distribution     3566 non-null   category
 8   assessment            3566 non-null   category
 9   pathology             3566 non-null   category
 10  subtlety              3566 non-null   category
 11  fileset               3566 non-null   object  
 12  mass_shape            3566 non-null   category
 13  mass_margins          3566 non-null   category
 14  case_id               3566 non-null   object  
 15  canc

,patient_id,breast_density,left_or_right_breast,image_view,abnormality_id,abnormality_type,calc_type,calc_distribution,assessment,pathology,subtlety,fileset,mass_shape,mass_margins,case_id,cancer
0,P_00005,3,RIGHT,CC,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,train,NOT APPLICABLE,NOT APPLICABLE,P_00005_RIGHT_calcification_CC_1,True
1,P_00005,3,RIGHT,MLO,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,train,NOT APPLICABLE,NOT APPLICABLE,P_00005_RIGHT_calcification_MLO_1,True
2,P_00007,4,LEFT,CC,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,train,NOT APPLICABLE,NOT APPLICABLE,P_00007_LEFT_calcification_CC_1,False
3,P_00007,4,LEFT,MLO,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,train,NOT APPLICABLE,NOT APPLICABLE,P_00007_LEFT_calcification_MLO_1,False
4,P_00008,1,LEFT,CC,1,calcification,AMORPHOUS,REGIONAL,2,BENIGN_WITHOUT_CALLBACK,3,train,NOT APPLICABLE,NOT APPLICABLE,P_00008_LEFT_calcification_CC_1,False


In [11]:
os.makedirs(os.path.dirname(FP_CLEAN_CASES), exist_ok=True)
df_clean.to_csv(FP_CLEAN_CASES, index=False)
df_calc = df_clean[CALC_VARIABLES].loc[df_clean['abnormality_type'] == 'calcification']
df_mass = df_clean[MASS_VARIABLES].loc[df_clean['abnormality_type'] == 'mass']
df_calc.to_csv(FP_CLEAN_CALC, index=False)
df_mass.to_csv(FP_CLEAN_MASS, index=False)

## Task 7: Random Sample Inspection

Let's take a look at a random sampling of the missing data and compare.

In [12]:
sample_cases = df_missing['case_id'].sample(10)
df_missing.loc[df_missing['case_id'].isin(sample_cases)]
df_clean.loc[df_clean['case_id'].isin(sample_cases)]

,patient_id,breast_density,left_or_right_breast,image_view,abnormality_id,abnormality_type,calc_type,calc_distribution,assessment,pathology,subtlety,fileset,mass_shape,mass_margins,case_id,cancer
263,P_00385,2.00,LEFT,CC,4,calcification,VASCULAR-COARSE-LUCENT_CENTERED,NaN,2,BENIGN_WITHOUT_CALLBACK,5,train,NOT APPLICABLE,NOT APPLICABLE,P_00385_LEFT_calcification_CC_4,False
402,P_00529,3.00,LEFT,MLO,1,calcification,LUCENT_CENTERED,NaN,2,BENIGN_WITHOUT_CALLBACK,3,train,NOT APPLICABLE,NOT APPLICABLE,P_00529_LEFT_calcification_MLO_1,False
646,P_00812,2.00,LEFT,CC,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,4,train,NOT APPLICABLE,NOT APPLICABLE,P_00812_LEFT_calcification_CC_1,False
975,P_01274,2.00,LEFT,MLO,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,train,NOT APPLICABLE,NOT APPLICABLE,P_01274_LEFT_calcification_MLO_1,False
977,P_01274,2.00,RIGHT,CC,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,4,train,NOT APPLICABLE,NOT APPLICABLE,P_01274_RIGHT_calcification_CC_1,False
1169,P_01457,2.00,RIGHT,MLO,1,calcification,COARSE,NaN,2,BENIGN_WITHOUT_CALLBACK,3,train,NOT APPLICABLE,NOT APPLICABLE,P_01457_RIGHT_calcification_MLO_1,False
1394,P_01823,3.00,RIGHT,MLO,3,calcification,LUCENT_CENTERED,NaN,2,BENIGN_WITHOUT_CALLBACK,4,train,NOT APPLICABLE,NOT APPLICABLE,P_01823_RIGHT_calcification_MLO_3,False
1572,P_00140,2.00,RIGHT,MLO,2,calcification,PUNCTATE,NaN,2,BENIGN_WITHOUT_CALLBACK,5,test,NOT APPLICABLE,NOT APPLICABLE,P_00140_RIGHT_calcification_MLO_2,False
2411,P_00775,2.00,LEFT,MLO,1,mass,NOT APPLICABLE,NOT APPLICABLE,3,BENIGN_WITHOUT_CALLBACK,5,train,ASYMMETRIC_BREAST_TISSUE,NaN,P_00775_LEFT_mass_MLO_1,False
3005,P_01635,2.00,RIGHT,MLO,1,mass,NOT APPLICABLE,NOT APPLICABLE,5,BENIGN_WITHOUT_CALLBACK,5,train,ARCHITECTURAL_DISTORTION,NaN,P_01635_RIGHT_mass_MLO_1,False


,patient_id,breast_density,left_or_right_breast,image_view,abnormality_id,abnormality_type,calc_type,calc_distribution,assessment,pathology,subtlety,fileset,mass_shape,mass_margins,case_id,cancer
263,P_00385,2,LEFT,CC,4,calcification,VASCULAR-COARSE-LUCENT_CENTERED,SEGMENTAL,2,BENIGN_WITHOUT_CALLBACK,5,train,NOT APPLICABLE,NOT APPLICABLE,P_00385_LEFT_calcification_CC_4,False
402,P_00529,3,LEFT,MLO,1,calcification,LUCENT_CENTERED,SEGMENTAL,2,BENIGN_WITHOUT_CALLBACK,3,train,NOT APPLICABLE,NOT APPLICABLE,P_00529_LEFT_calcification_MLO_1,False
646,P_00812,2,LEFT,CC,1,calcification,VASCULAR,SEGMENTAL,2,BENIGN_WITHOUT_CALLBACK,4,train,NOT APPLICABLE,NOT APPLICABLE,P_00812_LEFT_calcification_CC_1,False
975,P_01274,2,LEFT,MLO,1,calcification,VASCULAR,SEGMENTAL,2,BENIGN_WITHOUT_CALLBACK,5,train,NOT APPLICABLE,NOT APPLICABLE,P_01274_LEFT_calcification_MLO_1,False
977,P_01274,2,RIGHT,CC,1,calcification,VASCULAR,SEGMENTAL,2,BENIGN_WITHOUT_CALLBACK,4,train,NOT APPLICABLE,NOT APPLICABLE,P_01274_RIGHT_calcification_CC_1,False
1169,P_01457,2,RIGHT,MLO,1,calcification,COARSE,SEGMENTAL,2,BENIGN_WITHOUT_CALLBACK,3,train,NOT APPLICABLE,NOT APPLICABLE,P_01457_RIGHT_calcification_MLO_1,False
1394,P_01823,3,RIGHT,MLO,3,calcification,LUCENT_CENTERED,SEGMENTAL,2,BENIGN_WITHOUT_CALLBACK,4,train,NOT APPLICABLE,NOT APPLICABLE,P_01823_RIGHT_calcification_MLO_3,False
1572,P_00140,2,RIGHT,MLO,2,calcification,PUNCTATE,SEGMENTAL,2,BENIGN_WITHOUT_CALLBACK,5,test,NOT APPLICABLE,NOT APPLICABLE,P_00140_RIGHT_calcification_MLO_2,False
2409,P_00775,2,LEFT,MLO,1,mass,NOT APPLICABLE,NOT APPLICABLE,3,BENIGN_WITHOUT_CALLBACK,5,train,ASYMMETRIC_BREAST_TISSUE,SPICULATED,P_00775_LEFT_mass_MLO_1,False
3003,P_01635,2,RIGHT,MLO,1,mass,NOT APPLICABLE,NOT APPLICABLE,5,BENIGN_WITHOUT_CALLBACK,5,train,ARCHITECTURAL_DISTORTION,SPICULATED,P_01635_RIGHT_mass_MLO_1,False


## Task 8: Data Quality Analysis 2.0
Ok, let's have another go at the data quality analysis.

### Completeness Analysis

In [13]:
dqc = CalcCaseDQA(filepath=FP_CLEAN_CALC)
dqm = MassCaseDQA(filepath=FP_CLEAN_MASS)
dqcc = dqc.analyze_completeness()
dqmc = dqm.analyze_completeness()

In [14]:
print(dqcc.summary)
print(dqmc.summary)



                          Completeness                          
                         Dataset | Calc Cases
                        Filename | calc_cases.csv
                         Records | 1872
                Complete Records | 1872
             Record Completeness | 1.0
                     Data Values | 26208
            Complete Data Values | 26208
         Data Value Completeness | 1.0




                          Completeness                          
                         Dataset | Mass Cases
                        Filename | mass_cases.csv
                         Records | 1694
                Complete Records | 1694
             Record Completeness | 1.0
                     Data Values | 23716
            Complete Data Values | 23716
         Data Value Completeness | 1.0




We're complete.

### Uniqueness Analysis

In [15]:
dqcu = dqc.analyze_uniqueness()
dqmu = dqm.analyze_uniqueness()
print(dqcu.summary)
print(dqmu.summary)



                           Uniqueness                           
                         Dataset | Calc Cases
                        Filename | calc_cases.csv
                         Records | 1872
                  Unique Records | 1872
               Record Uniqueness | 1.0
                     Data Values | 26208
              Unique Data Values | 2709
           Data Value Uniqueness | 0.103




                           Uniqueness                           
                         Dataset | Mass Cases
                        Filename | mass_cases.csv
                         Records | 1694
                  Unique Records | 1694
               Record Uniqueness | 1.0
                     Data Values | 23716
              Unique Data Values | 2658
           Data Value Uniqueness | 0.112




We're unique.

### Validity Analysis

In [16]:
dqcv = dqc.analyze_validity()
dqmv = dqm.analyze_validity()
print(dqcv.summary)
print(dqmv.summary)



                            Validity                            
                         Dataset | Calc Cases
                        Filename | calc_cases.csv
                         Records | 1872
                   Valid Records | 1827
                 Record Validity | 0.976
                     Data Values | 26208
               Valid Data Values | 26163
             Data Value Validity | 0.998




                            Validity                            
                         Dataset | Mass Cases
                        Filename | mass_cases.csv
                         Records | 1694
                   Valid Records | 1694
                 Record Validity | 1.0
                     Data Values | 23716
               Valid Data Values | 23716
             Data Value Validity | 1.0




We're valid. This concludes the data cleaning exercise.